<a href="https://colab.research.google.com/github/ERGOWHO/cs677/blob/main/csci677_assignment_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this assignment you will practice putting together a simple image classification pipeline based on the k-Nearest Neighbor or the SVM/Softmax classifier for [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset. The goals of this assignment are as follows:



*   Understand the basic Image Classification pipeline and the data-driven approach (train/predict stages).
*   Understand the train/val/test splits and the use of validation data for hyperparameter tuning.
*   Implement and apply a k-Nearest Neighbor (kNN) classifier.
*   Implement and apply a Multiclass Support Vector Machine (SVM) classifier.
*   Implement and apply a Softmax classifier.
*   Understand the differences and tradeoffs between these classifiers.

Please fill in all the **TODO** code blocks. Once you are ready to submit:

* Export the notebook `CSCI677_assignment_1.ipynb` as a PDF `[Your USC ID]_CSCI677_assignment_1.pdf`
* Submit your PDF file through [Blackboard](https://blackboard.usc.edu/)

Please make sure that the notebook have been run before exporting PDF, and your code and all cell outputs are visible in the your submitted PDF. Regrading request will not be accepted if your code/output is not visible in the original submission. Thank you!

# **Data Preparation**

[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) is a well known dataset composed of 60,000 colored 32x32 images. The utility function `cifar10()` returns the entire CIFAR-10 dataset as a set of four Torch tensors:
* `x_train` contains all training images (real numbers in the range  [0,1] )
* `y_train` contains all training labels (integers in the range  [0,9] )
* `x_test` contains all test images
* `y_test` contains all test labels

This function automatically downloads the CIFAR-10 dataset the first time you run it.

In [ ]:
import os
import torch
from torchvision.datasets import CIFAR10

def _extract_tensors(dset, num=None):
    x = torch.tensor(dset.data, dtype=torch.float32).permute(0, 3, 1, 2).div_(255)
    y = torch.tensor(dset.targets, dtype=torch.int64)
    if num is not None:
        if num <= 0 or num > x.shape[0]:
          raise ValueError('Invalid value num=%d; must be in the range [0, %d]'
                          % (num, x.shape[0]))
        x = x[:num].clone()
        y = y[:num].clone()
    return x, y

def cifar10(num_train=None, num_test=None):
    download = not os.path.isdir('cifar-10-batches-py')
    dset_train = CIFAR10(root='.', download=download, train=True)
    dset_test = CIFAR10(root='.', train=False)
    x_train, y_train = _extract_tensors(dset_train, num_train)
    x_test, y_test = _extract_tensors(dset_test, num_test)

    return x_train, y_train, x_test, y_test

Our data is going to be stored simply in the four variables: `x_train`, `x_test`, `y_train`, and `y_test`.


*   Training set: `x_train` is composed of 50,000 images where `y_train` references the corresponding labels.
*   Testing set: `x_test` is composed of 10,000 images where `y_test` references the corresponding labels.

In [ ]:
x_train, y_train, x_test, y_test = cifar10()

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# k-Nearest Neighbor (kNN) (30 pts)


## **Subsampling**

When implementing machine learning algorithms, it's usually a good idea to use a small sample of the full dataset. This way your code will run much faster, allowing for more interactive and efficient development. Once you are satisfied that you have correctly implemented the algorithm, you can then rerun with the entire dataset.

In [ ]:
# Subsample size
num_train = 500
num_test = 250

# Redeclaring x_train...y_test with subsample
x_train, y_train, x_test, y_test = cifar10(num_train, num_test)


## Compute Distance (10 pts)

Now that we have examined and prepared our data, it is time to implement the kNN classifier. We can break the process down into two steps:
1. Compute the (squared Euclidean) distances between all training examples and all test examples
2. Given these distances, for each test example find its k nearest neighbors and have them vote for the label to output

**NOTE**: When implementing algorithms in PyTorch, it's best to avoid loops in Python if possible. Instead it is preferable to implement your computation so that all loops happen inside PyTorch functions. This will usually be much faster than writing your own loops in Python, since PyTorch functions can be internally optimized to iterate efficiently, possibly using multiple threads. This is especially important when using a GPU to accelerate your code.

In [ ]:
def compute_distances(x_train, x_test):
    """
    Inputs:
    x_train: shape (num_train, C, H, W) tensor.
    x_test: shape (num_test, C, H, W) tensor.

    Returns:
    dists: shape (num_train, num_test) tensor where dists[j, i] is the
        Euclidean distance between the ith training image and the jth test
        image.
    """

    # Get the number of training and testing images
    num_train = x_train.shape[0]
    num_test = x_test.shape[0]

    # dists will be the tensor housing all distance measurements between testing and training
    dists = x_train.new_zeros(num_train, num_test)

    # Flatten tensors
    train = x_train.flatten(1)
    test = x_test.flatten(1)

    #######################################################################
    # TODO (10 pts):
    # find the Euclidean distance between testing and training images,
    # and save the computed distance in dists.
    #######################################################################

    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    train_norms = (train * train).sum(dim=1).reshape(-1, 1)
    test_norms = (test * test).sum(dim=1)
    dists = train_norms + test_norms - 2 * train.mm(test.t())
    dists = torch.sqrt(torch.clamp(dists, min=0.0))
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    return dists

## Implement kNN (10 pts)

The kNN classifier consists of two stages:

*   Training: the classifier takes the training data and simply remembers it
*   Testing: kNN classifies every test image by comparing to all training images and transfering the labels of the k most similar training examples

In [ ]:
class KnnClassifier:
    def __init__(self, x_train, y_train):
        """
        x_train: shape (num_train, C, H, W) tensor where num_train is batch size,
          C is channel size, H is height, and W is width.
        y_train: shape (num_train) tensor where num_train is batch size providing labels
        """

        self.x_train = x_train
        self.y_train = y_train

    def predict(self, x_test, k=1):
        """
        x_test: shape (num_test, C, H, W) tensor where num_test is batch size,
          C is channel size, H is height, and W is width.
        k: The number of neighbors to use for prediction
        """

        # Init output shape
        y_test_pred = torch.zeros(x_test.shape[0], dtype=torch.int64)

        # Find & store Euclidean distance between test & train
        dists = compute_distances(self.x_train, x_test)

        #######################################################################
        # TODO (10 pts):
        # The goal is to return a tensor y_test_pred where the ith index
        # is the assigned label to ith test image by the kNN algorithm.
        #######################################################################

        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        for i in range(x_test.shape[0]):
          k_nearest_neighbors = dists[:, i].argsort()[:k]
          k_nearest_labels = self.y_train[k_nearest_neighbors]
          y_test_pred[i] = k_nearest_labels.mode()[0]
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        return y_test_pred

    def check_accuracy(self, x_test, y_test, k=1, quiet=False):
        """
        x_test: shape (num_test, C, H, W) tensor where num_test is batch size,
          C is channel size, H is height, and W is width.
        y_test: shape (num_test) tensor where num_test is batch size providing labels
        k: The number of neighbors to use for prediction
        quiet: If True, don't print a message.

        Returns:
        accuracy: Accuracy of this classifier on the test data, as a percent.
          Python float in the range [0, 100]
        """

        y_test_pred = self.predict(x_test, k=k)
        num_samples = x_test.shape[0]
        num_correct = (y_test == y_test_pred).sum().item()
        accuracy = 100.0 * num_correct / num_samples
        msg = (f'Got {num_correct} / {num_samples} correct; '
              f'accuracy is {accuracy:.2f}%')
        if not quiet:
          print(msg)
        return accuracy

We've finished implementing kNN and can begin testing the algorithm on larger portions of the dataset to see how well it performs.

In [ ]:
torch.manual_seed(0)
num_train = 5000
num_test = 500
x_train, y_train, x_test, y_test = cifar10(num_train, num_test)

classifier = KnnClassifier(x_train, y_train)
classifier.check_accuracy(x_test, y_test, k=5)

Got 139 / 500 correct; accuracy is 27.80%


27.8

## Cross-validation (10 pts)

As our algorithm currently exists, we have to manually tune the hyperparameter $k$ to some integer value, which raises the question - is that the best value for $k$? Cross validation is a procedure to automate selecting an optimal value for $k$.



In [ ]:
def knn_cross_validate(x_train, y_train, num_folds=5, k_choices=None):
    """
    Inputs:
    x_train: Tensor of shape (num_train, C, H, W) giving all training data
    y_train: int64 tensor of shape (num_train,) giving labels for training data
    num_folds: Integer giving the number of folds to use
    k_choices: List of integers giving the values of k to try

    Returns:
    k_to_accuracies: Dictionary mapping values of k to lists, where
        k_to_accuracies[k][i] is the accuracy on the ith fold of a KnnClassifier
        that uses k nearest neighbors.
    """
    # Create a list of k's for testing
    if k_choices is None:
        k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

    # Create empty lists to house the chunks for cross validation
    x_train_folds = []
    y_train_folds = []

    # Flatten x_train from [5000, 3, 32, 32] to [5000, 3072]
    x_train_flat = x_train.view(x_train.shape[0], -1)

    # Partition our training set to 5 tensors of training images of shape [1000, 3072] and 5 labels of shape [1000]
    x_train_folds = torch.chunk(x_train_flat, num_folds, dim=0)
    y_train_folds = torch.chunk(y_train, num_folds, dim=0)

    # Create object to house the combination of accuracies for different values k for different validation sets
    k_to_accuracies = {}

    #######################################################################
    # TODO (10 pts):
    # Iterate through every combination of k_choices and possible validation sets
    # Return k_to_accuracies: Dictionary mapping values of k to lists
    #######################################################################

    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    for k in k_choices:
        k_to_accuracies[k] = []

        for i in range(num_folds):
            x_train_tmp = torch.cat([fold for j, fold in enumerate(x_train_folds) if j != i])
            y_train_tmp = torch.cat([fold for j, fold in enumerate(y_train_folds) if j != i])

            x_val_tmp = x_train_folds[i]
            y_val_tmp = y_train_folds[i]


            classifier = KnnClassifier(x_train_tmp.view(-1, 3, 32, 32), y_train_tmp)

            accuracy = classifier.check_accuracy(x_val_tmp.view(-1, 3, 32, 32), y_val_tmp, k=k, quiet=True)
            k_to_accuracies[k].append(accuracy)
    return k_to_accuracies
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


def knn_get_best_k(k_to_accuracies):
    """
    Inputs:
    - k_to_accuracies: Dictionary mapping values of k to lists, where
    k_to_accuracies[k][i] is the accuracy on the ith fold of a KnnClassifier
    that uses k nearest neighbors.

    Returns:
    - best_k: best (and smallest if there is a conflict) k value based on
    the k_to_accuracies info
    """
    # Create best_k variable to return optimal k
    best_k = 0

    # Get keys and values from k_to_accuracies object
    keys = [k for k in k_to_accuracies.keys()]
    values = [v for v in k_to_accuracies.values()]

    # Get largest average of all the values
    max_avg = torch.argmax(torch.mean(torch.tensor(values), dim=1))
    # Get corresponding k for max_avg
    best_k = keys[max_avg]

    return best_k


Now we can use the results of cross-validation to select the best value for $k$, and rerun the classifier on our full 5000 set of training examples.

In [ ]:
torch.manual_seed(0)

k_to_accuracies = knn_cross_validate(x_train, y_train, num_folds=5)

for k, accs in sorted(k_to_accuracies.items()):
  print('k = %d got accuracies: %r' % (k, accs))

best_k = knn_get_best_k(k_to_accuracies)
print('Best k is ', best_k)

classifier = KnnClassifier(x_train, y_train)
classifier.check_accuracy(x_test, y_test, k=best_k)

k = 1 got accuracies: [26.3, 25.7, 26.4, 27.8, 26.6]
k = 3 got accuracies: [23.9, 24.9, 24.0, 26.6, 25.4]
k = 5 got accuracies: [24.8, 26.6, 28.0, 29.2, 28.0]
k = 8 got accuracies: [26.2, 28.2, 27.3, 29.0, 27.3]
k = 10 got accuracies: [26.5, 29.6, 27.6, 28.4, 28.0]
k = 12 got accuracies: [26.0, 29.5, 27.9, 28.3, 28.0]
k = 15 got accuracies: [25.2, 28.9, 27.8, 28.2, 27.4]
k = 20 got accuracies: [27.0, 27.9, 27.9, 28.2, 28.5]
k = 50 got accuracies: [27.1, 28.8, 27.8, 26.9, 26.6]
k = 100 got accuracies: [25.6, 27.0, 26.3, 25.6, 26.3]
Best k is  10
Got 141 / 500 correct; accuracy is 28.20%


28.2

# Define a General Classifier Class (20 pts)

Before implementing Support Vector Machine (SVM) and Softmax Classifier. We define a general classifier class that contains the following main functions:


1.   `train`: train this linear classifier using stochastic gradient descent.
2.   `predict`: use the trained weights of this linear classifier to predict labels for data points.
3.   `loss`: compute the loss function and its derivative.

We will define SVM and Softmax classifier as subclasses of this general linear classifier class. Subclasses will override the `loss` function.





In [ ]:
import numpy as np

class LinearClassifier(object):
    def __init__(self):
        self.W = None

    def train(
        self,
        X,
        y,
        learning_rate=1e-3,
        reg=1e-5,
        num_iters=100,
        batch_size=200,
        verbose=False,
    ):
        """
        Train this linear classifier using stochastic gradient descent.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.
        - y: A numpy array of shape (N,) containing training labels; y[i] = c
          means that X[i] has label 0 <= c < C for C classes.
        - learning_rate: (float) learning rate for optimization.
        - reg: (float) regularization strength.
        - num_iters: (integer) number of steps to take when optimizing
        - batch_size: (integer) number of training examples to use at each step.
        - verbose: (boolean) If true, print progress during optimization.

        Outputs:
        A list containing the value of the loss function at each training iteration.
        """
        num_train, dim = X.shape
        num_classes = (
            np.max(y) + 1
        )  # assume y takes values 0...K-1 where K is number of classes
        if self.W is None:
            # lazily initialize W
            self.W = 0.001 * np.random.randn(dim, num_classes)

        # Run stochastic gradient descent to optimize W
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            #########################################################################
            # TODO (10 pts):                                                        #
            # Sample batch_size elements from the training data and their           #
            # corresponding labels to use in this round of gradient descent.        #
            # Store the data in X_batch and their corresponding labels in           #
            # y_batch; after sampling X_batch should have shape (batch_size, dim)   #
            # and y_batch should have shape (batch_size,)                           #
            #                                                                       #
            # Hint: Use np.random.choice to generate indices. Sampling with         #
            # replacement is faster than sampling without replacement.              #
            #########################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            indices = np.random.choice(num_train, batch_size, replace=True)
            X_batch = X[indices]
            y_batch = y[indices]

            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            # evaluate loss and gradient
            loss, grad = self.loss(X_batch, y_batch, reg)
            loss_history.append(loss)

            # perform parameter update
            #########################################################################
            # TODO (5 pts):                                                         #
            # Update the weights using the gradient and the learning rate.          #
            #########################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            self.W -= learning_rate * grad
            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            if verbose and it % 100 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        """
        Use the trained weights of this linear classifier to predict labels for
        data points.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.

        Returns:
        - y_pred: Predicted labels for the data in X. y_pred is a 1-dimensional
          array of length N, and each element is an integer giving the predicted
          class.
        """
        y_pred = np.zeros(X.shape[0])
        ###########################################################################
        # TODO (5 pts):                                                           #
        # Implement this method. Store the predicted labels in y_pred.            #
        ###########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        scores = X.dot(self.W)
        y_pred = np.argmax(scores, axis=1)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return y_pred

    def loss(self, X_batch, y_batch, reg):
        """
        Compute the loss function and its derivative.
        Subclasses will override this.

        Inputs:
        - X_batch: A numpy array of shape (N, D) containing a minibatch of N
          data points; each point has dimension D.
        - y_batch: A numpy array of shape (N,) containing labels for the minibatch.
        - reg: (float) regularization strength.

        Returns: A tuple containing:
        - loss as a single float
        - gradient with respect to self.W; an array of the same shape as W
        """
        pass

# Multiclass Support Vector Machine (SVM) (30 pts)



[Support vector machines (SVMs)](https://scikit-learn.org/stable/modules/svm.html) are a set of supervised learning methods used for classification.

The advantages of support vector machines are:

* Effective in high dimensional spaces.
* Still effective in cases where number of dimensions is greater than the number of samples.
* Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
* Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

* If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.
* SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

In this section, we will first implement the loss function for SVM and use the validation set to tune hyperparameters.

**NOTE:** please use [numpy](https://numpy.org/), please do not use [scikit-learn](https://scikit-learn.org/stable/), [PyTorch](https://pytorch.org/) or other libraries.

## Loss Function (20 pts)

We first structure the loss function for SVM. For detailed explanations of SVM loss, please check out [this reading material](https://cs231n.github.io/linear-classify/#loss-function).

In [ ]:
import time
import numpy as np

def svm_loss(W, X, y, reg):
    """
    Structured SVM loss function implementation.

    Inputs have dimension D, there are C classes, and we operate on minibatches
    of N examples.

    Inputs:
    - W: A numpy array of shape (D, C) containing weights.
    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c means
      that X[i] has label c, where 0 <= c < C.
    - reg: (float) regularization strength

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; an array of same shape as W
    """
    loss = 0.0
    dW = np.zeros(W.shape)  # initialize the gradient as zero

    #############################################################################
    # TODO (10 pts):                                                            #
    # Implement a vectorized version of the structured SVM loss, storing the    #
    # result in loss.                                                           #
    #############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    num_train = X.shape[0]

    scores = X.dot(W)

    correct_scores = scores[np.arange(num_train), y]
    correct_scores = correct_scores[:, np.newaxis]

    margins = np.maximum(0, scores - correct_scores + 1)

    margins[np.arange(num_train), y] = 0

    loss = np.sum(margins) / num_train
    loss += 0.5 * reg * np.sum(W * W)
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    #############################################################################
    # TODO (10 pts):                                                            #
    # Implement a vectorized version of the gradient for the structured SVM     #
    # loss, storing the result in dW.                                           #
    #                                                                           #
    # Hint: Instead of computing the gradient from scratch, it may be easier    #
    # to reuse some of the intermediate values that you used to compute the     #
    # loss.                                                                     #
    #############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    binary_margins = margins
    binary_margins[margins > 0] = 1
    row_sum = np.sum(binary_margins, axis=1)
    binary_margins[np.arange(num_train), y] = -row_sum
    dW = X.T.dot(binary_margins) / num_train + reg * W

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    return loss, dW


Before we test our implementation of SVM loss function, we need to first convert the previously loaded CIFAR-10 dataset from PyTorch tensors into NumPy array, split the data into train, val and test sets, and preprocess the images.

In [ ]:
torch.manual_seed(0)
num_train = 50000
num_test = 5000
x_train, y_train, x_test, y_test = cifar10(num_train, num_test)

# Split the data into train, val, and test sets. In addition we will
# create a small development set as a subset of the training data;
# we can use this for development so our code runs faster.
num_training = 49000
num_validation = 1000
num_test = 1000
num_dev = 500

x_train_np = x_train.numpy()
y_train_np = y_train.numpy()
x_test_np = x_test.numpy()
y_test_np = y_test.numpy()

# Our validation set will be num_validation points from the original
# training set.
mask = range(num_training, num_training + num_validation)
X_val = x_train_np[mask]
y_val = y_train_np[mask]

# Our training set will be the first num_train points from the original
# training set.
mask = range(num_training)
X_train = x_train_np[mask]
y_train = y_train_np[mask]

# We will also make a development set, which is a small subset of
# the training set.
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = x_train_np[mask]
y_dev = y_train_np[mask]

# We use the first num_test points of the original test set as our
# test set.
mask = range(num_test)
X_test = x_test_np[mask]
y_test = y_test_np[mask]

# Preprocessing: reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

# As a sanity check, print out the shapes of the data
print('Training data shape: ', X_train.shape)
print('Validation data shape: ', X_val.shape)
print('Test data shape: ', X_test.shape)
print('dev data shape: ', X_dev.shape)

# Preprocessing: subtract the mean image
# first: compute the image mean based on the training data
mean_image = np.mean(X_train, axis=0)

# second: subtract the mean image from train and test data
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

# third: append the bias dimension of ones (i.e. bias trick) so that our SVM
# only has to worry about optimizing a single weight matrix W.
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

Training data shape:  (49000, 3072)
Validation data shape:  (1000, 3072)
Test data shape:  (1000, 3072)
dev data shape:  (500, 3072)
(49000, 3073) (1000, 3073) (1000, 3073) (500, 3073)


Now, we can test our implementation of SVM loss.

In [ ]:
# generate a random SVM weight matrix of small numbers
W = np.random.randn(3073, 10) * 0.0001

tic = time.time()
loss, _ = svm_loss(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('loss: %e computed in %fs' % (loss, toc - tic))

loss: 9.000802e+00 computed in 0.017666s


In [ ]:
class LinearSVM(LinearClassifier):
    """ A subclass that uses the Multiclass SVM loss function """

    def loss(self, X_batch, y_batch, reg):
        return svm_loss(self.W, X_batch, y_batch, reg)

In [ ]:

svm = LinearSVM()
tic = time.time()
loss_hist = svm.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4,
                      num_iters=1500, verbose=True)
toc = time.time()
print('That took %fs' % (toc - tic))


iteration 0 / 1500: loss 390.063601
iteration 100 / 1500: loss 239.977485
iteration 200 / 1500: loss 149.006800
iteration 300 / 1500: loss 93.863941
iteration 400 / 1500: loss 60.443459
iteration 500 / 1500: loss 40.181883
iteration 600 / 1500: loss 27.902043
iteration 700 / 1500: loss 20.456265
iteration 800 / 1500: loss 15.940348
iteration 900 / 1500: loss 13.207344
iteration 1000 / 1500: loss 11.548549
iteration 1100 / 1500: loss 10.542665
iteration 1200 / 1500: loss 9.934569
iteration 1300 / 1500: loss 9.564847
iteration 1400 / 1500: loss 9.342052
That took 8.360579s


In [ ]:
y_train_pred = svm.predict(X_train)
print('training accuracy: %f' % (np.mean(y_train == y_train_pred), ))
y_val_pred = svm.predict(X_val)
print('validation accuracy: %f' % (np.mean(y_val == y_val_pred), ))

training accuracy: 0.231082
validation accuracy: 0.248000


## Hyperparameter Tuning (10 pts)

Now we use the validation set to tune hyperparameters (regularization strength and learning rate). You should experiment with different ranges for the learning rates and regularization strengths; if you are careful you should be able to get a classification accuracy of about 0.39 (> 0.385) on the validation set.

**Note:** you may see runtime/overflow warnings during hyper-parameter search. This may be caused by extreme values, and is not a bug.

In [ ]:

# results is dictionary mapping tuples of the form
# (learning_rate, regularization_strength) to tuples of the form
# (training_accuracy, validation_accuracy). The accuracy is simply the fraction
# of data points that are correctly classified.
results = {}
best_val = -1   # The highest validation accuracy that we have seen so far.
best_svm = None # The LinearSVM object that achieved the highest validation rate.

################################################################################
# TODO (10 pts):                                                               #
# Write code that chooses the best hyperparameters by tuning on the validation #
# set. For each combination of hyperparameters, train a linear SVM on the      #
# training set, compute its accuracy on the training and validation sets, and  #
# store these numbers in the results dictionary. In addition, store the best   #
# validation accuracy in best_val and the LinearSVM object that achieves this  #
# accuracy in best_svm.                                                        #
#                                                                              #
# Hint: You should use a small value for num_iters as you develop your         #
# validation code so that the SVMs don't take much time to train; once you are #
# confident that your validation code works, you should rerun the validation   #
# code with a larger value for num_iters.                                      #
################################################################################

# Provided as a reference. You may or may not want to change these hyperparameters
learning_rates = [0.01]
regularization_strengths = [0.1]


# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

for lr in learning_rates:
    for reg in regularization_strengths:
        svm = LinearSVM()
        svm.train(X_train, y_train, learning_rate=lr, reg=reg, num_iters=5000)

        y_train_pred = svm.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)

        y_val_pred = svm.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)

        results[(lr, reg)] = (train_accuracy, val_accuracy)

        if val_accuracy > best_val:
            best_val = val_accuracy
            best_svm = svm

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))

print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-02 reg 1.000000e-01 train accuracy: 0.397245 val accuracy: 0.406000
best validation accuracy achieved during cross-validation: 0.406000


# Softmax Classifier (20 pts)

In this section, we will first implement the loss function for softmax classifier, and then use the validation set to set the learning rate and regularization strength.

## Loss Function (10 pts)

In [ ]:
def softmax_loss(W, X, y, reg):
    """
    Softmax loss function

    Inputs have dimension D, there are C classes, and we operate on minibatches
    of N examples.

    Inputs:
    - W: A numpy array of shape (D, C) containing weights.
    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c means
      that X[i] has label c, where 0 <= c < C.
    - reg: (float) regularization strength

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; an array of same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # TODO (10 pts):                                                            #
    # Compute the softmax loss and its gradient using no explicit loops.        #
    # Store the loss in loss and the gradient in dW. If you are not careful     #
    # here, it is easy to run into numeric instability. Don't forget the        #
    # regularization!                                                           #
    #############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    scores = X.dot(W)
    scores -= np.max(scores, axis=1, keepdims=True)

    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    correct_log_probs = -np.log(probs[np.arange(X.shape[0]), y])
    loss = np.sum(correct_log_probs) / X.shape[0]

    loss += 0.5 * reg * np.sum(W * W)

    dscores = probs
    dscores[np.arange(X.shape[0]), y] -= 1
    dscores /= X.shape[0]

    dW = X.T.dot(dscores)
    dW += reg * W

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    return loss, dW

In [ ]:
class Softmax(LinearClassifier):
    """ A subclass that uses the Softmax + Cross-entropy loss function """

    def loss(self, X_batch, y_batch, reg):
        return softmax_loss(self.W, X_batch, y_batch, reg)

## Hyperparameter Tuning (10 pts)

In [ ]:
results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO (10 pts):                                                               #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################

# Provided as a reference. You may or may not want to change these hyperparameters
learning_rates = [0.009]
regularization_strengths = [0.09]
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
for lr in learning_rates:
    for reg in regularization_strengths:
        softmax_classifier = Softmax()

        loss_hist = softmax_classifier.train(X_train, y_train, learning_rate=lr, reg=reg,
                                             num_iters=1500, verbose=False)

        y_train_pred = softmax_classifier.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)

        y_val_pred = softmax_classifier.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)

        results[(lr, reg)] = (train_accuracy, val_accuracy)

        if val_accuracy > best_val:
            best_val = val_accuracy
            best_softmax = softmax_classifier
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))

print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 9.000000e-03 reg 9.000000e-02 train accuracy: 0.383980 val accuracy: 0.387000
best validation accuracy achieved during cross-validation: 0.387000


In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.386000


# Acknowledgement

Credits to [UMichigan's 498/598 Deep Learning for Computer Vision](https://web.eecs.umich.edu/~justincj/teaching/eecs498/FA2020/) and Stanfords's [CS231n: Convolutional Neural Networks for Visual Recognition](https://cs231n.github.io/), some code is adapted from their courses's assignments.